In [24]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of page to be scraped
url = 'https://www.indeed.com/jobs?q=data%20analyst&l=austin'
browser.visit(url)

In [5]:
title_list=[]
company_list=[]
location_list=[]
link_description_list=[]


for x in range(1, 75):
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_="jobsearch-SerpJobCard")
    
    
    for result in results:
    
        title=result.a['title']
        title_list.append(title)
        company=result.find('span', class_='company').text
        company_list.append(company)
        location=result.find(class_='location').text
        location_list.append(location)
        link_description=result.a['href']
        link_description_list.append(link_description)
        
        #print('page:', x, '-------------')
        #print(title)
        #print(company)
        #print(location)
        #print(link_description)
        #print("-------------------------")
        

    
    

In [6]:
description_url_list = ['http://indeed.com' + url for url in link_description_list]

In [7]:
description_text_list=[]
URL=[]

for urls in description_url_list:
    URL.append(urls)
    browser.visit(urls)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    description_results=soup.find_all('div', class_="jobsearch-JobComponent-description")
    for description in description_results:
        actual_description=description.text
    description_text_list.append(actual_description)
#         print(urls)
#         print(actual_description)
#         print('------------------------')

In [17]:
jobs_df = pd.DataFrame({
    "Job Title": title_list,
    "Company": company_list,
    "Locatoin": location_list,
    "Link Description": link_description_list
})

In [19]:
jobs_df.head()

,Job Title,Company,Locatoin,Link Description
0,Healthcare Data Analyst,\n\n TML MultiState IEBP,"Austin, TX 78754",/pagead/clk?mo=r&ad=-6NYlbfkN0D_AngAViu68QezII...
1,Data Analyst,\n\n Silvercar,"Austin, TX",/pagead/clk?mo=r&ad=-6NYlbfkN0ALhMgdRB8YT1nVMS...
2,Data Privacy Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
3,Digital Analyst,\n PivotalAnalytics,"Austin, TX",/pagead/clk?mo=r&ad=-6NYlbfkN0ALIVyn3YdKH-PRuH...
4,TRACS Data Analyst 06-19,\n\n Housing Authority of The City of A...,"Austin, TX 78704 (South Lamar-South Congress a...",/rc/clk?jk=c54e5625ad2987c2&fccid=314136e4f82b...


In [21]:
description_df = pd.DataFrame({
    "Description": description_text_list,
    "URL": URL
})

In [22]:
description_df.head()

,Description,URL
0,SUMMARY: TML Multistate IEBP is pleased to ann...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,About Silvercar:\nSilvercar's unique combinati...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,Our mission:\nAs the world’s number 1 job site...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,Company Overview:PivotalAnalytics is an analyt...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,$18.50 an hourJOB NOTICE\n DATE POSTED: Februa...,http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...


In [25]:
# Connect to local database
rds_connection_string = "root:KB@127.0.0.1/jobs_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [26]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [27]:
# Connect to local database
rds_connection_string = "root:KB@127.0.0.1/jobs_db"
engine = create_engine(f'mysql://{rds_connection_string}')

### Use pandas to load DataFrame into database

In [31]:
jobs_df.to_sql(name='job_postings', con=engine, if_exists='replace', index=False)

In [33]:
description_df.to_sql(name='description', con=engine, if_exists='replace', index=False)

### Query from SQL

In [32]:
pd.read_sql_query('select * from job_postings', con=engine).head()

,Job Title,Company,Locatoin,Link Description
0,Healthcare Data Analyst,\n\n TML MultiState IEBP,"Austin, TX 78754",/pagead/clk?mo=r&ad=-6NYlbfkN0D_AngAViu68QezII...
1,Data Analyst,\n\n Silvercar,"Austin, TX",/pagead/clk?mo=r&ad=-6NYlbfkN0ALhMgdRB8YT1nVMS...
2,Data Privacy Analyst,\n\n Indeed,"Austin, TX 78731",/pagead/clk?mo=r&ad=-6NYlbfkN0CiRNM7CVr8YueLFK...
3,Digital Analyst,\n PivotalAnalytics,"Austin, TX",/pagead/clk?mo=r&ad=-6NYlbfkN0ALIVyn3YdKH-PRuH...
4,TRACS Data Analyst 06-19,\n\n Housing Authority of The City of A...,"Austin, TX 78704 (South Lamar-South Congress a...",/rc/clk?jk=c54e5625ad2987c2&fccid=314136e4f82b...


In [34]:
pd.read_sql_query('select * from description', con=engine).head()

,Description,URL
0,SUMMARY: TML Multistate IEBP is pleased to ann...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
1,About Silvercar:\nSilvercar's unique combinati...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
2,Our mission:\nAs the world’s number 1 job site...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
3,Company Overview:PivotalAnalytics is an analyt...,http://indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN...
4,$18.50 an hourJOB NOTICE\n DATE POSTED: Februa...,http://indeed.com/rc/clk?jk=c54e5625ad2987c2&f...
